In [ ]:
import cv2
import numpy as np
import tensorflow as tf
tf.compat.v1.disable_eager_execution() # Speed
from tensorflow.keras.models import Model, load_model, model_from_json
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, \
                                    Conv2DTranspose, BatchNormalization, \
                                    UpSampling2D, Concatenate

In [ ]:
from tensorflow.keras.utils import Sequence
class DataGen(Sequence):
    def __init__(self, x, y, batch_size):
        self.x, self.y = x, y
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        x_batch = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        y_batch = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        x_ret, y_ret = [], []
        for x_file_name, y_file_name in zip(x_batch, y_batch):
            x_img = cv2.resize(cv2.imread(x_file_name, 0), (256, 256))
            y_img = cv2.resize(cv2.imread(y_file_name, cv2.IMREAD_UNCHANGED), (256, 256))
            probs = y_img[:,:,3] / 255.0
            alpha_map = np.zeros((*probs.shape, 2))
            alpha_map[:,:,0] = 1 - probs
            alpha_map[:,:,1] = probs
            x_ret.append(x_img[:,:,np.newaxis])
            y_ret.append(alpha_map)

        return np.array(x_ret), np.array(y_ret)

In [ ]:
def LikeUnet():
    # input
    im = Input(shape=(None, None, 1))
    # conv1
    x = Conv2D(16, 3, padding='same', activation='relu')(im)
    conv1 = MaxPooling2D(2)(x)
    # conv2
    x = Conv2D(32, 3, padding='same', activation='relu')(conv1)
    conv2 = MaxPooling2D(2)(x)
    # conv3
    x = Conv2D(64, 3, padding='same', activation='relu')(conv2)
    conv3 = MaxPooling2D(2)(x)
    # conv4
    x = Conv2D(128, 3, padding='same', activation='relu')(conv3)
    conv4 = MaxPooling2D(2)(x)
    # conv5
    x = Conv2D(256, 3, padding='same', activation='relu')(conv4)
    x = MaxPooling2D(2)(x)
    # BatchNormalization
    x = BatchNormalization()(x)
    # trans-conv1
    x = Conv2D(256, 3, padding='same', activation='relu')(x)
    x = Conv2DTranspose(128, 3, padding='same', activation='relu')(x)
    x = UpSampling2D(2, interpolation='bilinear')(x)
    # trans-conv2
    x = Concatenate()([conv4, x])
    x = Conv2D(128, 3, padding='same', activation='relu')(x)
    x = Conv2DTranspose(64, 3, padding='same', activation='relu')(x)
    x = UpSampling2D(2, interpolation='bilinear')(x)
    # trans-conv3
    x = Concatenate()([conv3, x])
    x = Conv2D(64, 3, padding='same', activation='relu')(x)
    x = Conv2DTranspose(32, 3, padding='same', activation='relu')(x)
    x = UpSampling2D(2, interpolation='bilinear')(x)
    # trans-conv4
    x = Concatenate()([conv2, x])
    x = Conv2D(32, 3, padding='same', activation='relu')(x)
    x = Conv2DTranspose(16, 3, padding='same', activation='relu')(x)
    x = UpSampling2D(2, interpolation='bilinear')(x)
    # trans-conv5
    x = Concatenate()([conv1, x])
    x = Conv2D(16, 3, padding='same', activation='relu')(x)
    x = Conv2DTranspose(8, 3, padding='same', activation='relu')(x)
    x = UpSampling2D(2, interpolation='bilinear')(x)
    # output
    x = Conv2D(4, 3, padding='same', activation='relu')(x)
    out = Conv2D(2, 1, padding='same', activation='softmax')(x)
    return Model(inputs=im, outputs=out)

In [ ]:
with open("x_train.txt", "r") as f:
    x_train = [filename[:-1] for filename in f.readlines()]
with open("y_train.txt", "r") as f:
    y_train = [filename[:-1] for filename in f.readlines()]
with open("x_val.txt", "r") as f:
    x_val = [filename[:-1] for filename in f.readlines()]
with open("y_val.txt", "r") as f:
    y_val = [filename[:-1] for filename in f.readlines()]

In [ ]:
trainGen = DataGen(x_train, y_train, batch_size=60)
valGen = DataGen(x_val, y_val, batch_size=60)

In [ ]:
from tensorflow.keras.metrics import MeanIoU
class mIoU(MeanIoU):
    def __init__(self, num_classes):
        super().__init__(num_classes=num_classes, name="mIoU")
    
    def __call__(self, y_true, y_pred, sample_weight=None):
        y_true = tf.argmax(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        return super().__call__(y_true, y_pred, sample_weight=sample_weight)

In [ ]:
# model = LikeUnet()
# with open('dark.json', 'w') as f:
#     f.write(model.to_json())
with open('dark.json', 'r') as f:
    json_string = f.read()
model = model_from_json(json_string)
model.load_weights('models\\dark-20-0.9774.h5')
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=[mIoU(num_classes=2)])

In [ ]:
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
logdir = "logs\\" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logdir, update_freq='batch', profile_batch=0)
filepath = "models\\dark-{epoch:02d}-{val_mIoU:.4f}.h5"
checkpoint_callback = ModelCheckpoint(filepath, monitor='val_mIoU', save_weights_only=True)

In [ ]:
# Small dataset for debugging
# model.fit_generator(generator=valGen, epochs=2, workers=8, shuffle=True)

In [ ]:
model.fit_generator(generator=trainGen, validation_data=valGen, epochs=30, workers=8, shuffle=True,
                    callbacks=[tensorboard_callback, checkpoint_callback], initial_epoch=20)